In [183]:
import numpy as np
import pandas as pd
from sklearn import preprocessing,metrics,manifold
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV,cross_val_predict
import matplotlib.patches as mpatches
from sklearn.metrics import accuracy_score
%matplotlib inline
from sklearn.preprocessing import RobustScaler
import xgboost
from sklearn.metrics import classification_report,roc_auc_score,roc_curve,r2_score,recall_score,confusion_matrix,precision_recall_curve
from collections import Counter
from sklearn.model_selection import StratifiedKFold,KFold,StratifiedShuffleSplit
from nltk import word_tokenize
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, TruncatedSVD,SparsePCA
from sklearn.metrics import classification_report,confusion_matrix
from nltk.tokenize import word_tokenize
from collections import defaultdict
from collections import Counter
import seaborn as sns
import keras
from keras.preprocessing.text import Tokenizer
import tensorflow as tf
from keras.layers import Dense, BatchNormalization
from tensorflow.keras.preprocessing.sequence import pad_sequences
from wordcloud import WordCloud,STOPWORDS
import nltk
from nltk.corpus import stopwords
import string
from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go


PRE - PROCESSING 

In [184]:
train_df = pd.read_csv('train.csv', encoding='latin-1')
train_df.head(400)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
395,ham,From here after The performance award is calcu...,NaN,NaN,NaN
396,ham,Was actually sleeping and still might when u c...,NaN,NaN,NaN
397,ham,You are always putting your business out there...,NaN,NaN,NaN
398,ham,"Good evening Sir, Al Salam Wahleykkum.sharing ...",NaN,NaN,NaN


In [185]:
train_df=train_df.drop(columns=["Unnamed: 2","Unnamed: 3","Unnamed: 4"])
train_df

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [186]:
train_df['v1']=train_df['v1'].map({'ham':0, 'spam':1})
train_df.head(20)

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
5,1,FreeMsg Hey there darling it's been 3 week's n...
6,0,Even my brother is not like to speak with me. ...
7,0,As per your request 'Melle Melle (Oru Minnamin...
8,1,WINNER!! As a valued network customer you have...
9,1,Had your mobile 11 months or more? U R entitle...


In [187]:
len(train_df),train_df.index.shape[-1]


(5572, 5572)

In [188]:
train_df = train_df.dropna()
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   int64 
 1   v2      5572 non-null   object
dtypes: int64(1), object(1)
memory usage: 87.2+ KB


In [189]:
dict_df={
    "spam":train_df["v1"],
    "message":train_df["v2"],
}
df = pd.DataFrame(dict_df)
df["message"] = df["message"].astype(str)
df.head()

,spam,message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


NATURAL LANGUAGE PROCESSING TECHNIQUES

In [190]:
#lowercasing 
#lowercasing 
df["text_lower"] = df["message"].str.lower()
df.head(10)

,spam,message,text_lower
0,0,"Go until jurong point, crazy.. Available only ...","go until jurong point, crazy.. available only ..."
1,0,Ok lar... Joking wif u oni...,ok lar... joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in 2 a wkly comp to win fa cup fina...
3,0,U dun say so early hor... U c already then say...,u dun say so early hor... u c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro...","nah i don't think he goes to usf, he lives aro..."
5,1,FreeMsg Hey there darling it's been 3 week's n...,freemsg hey there darling it's been 3 week's n...
6,0,Even my brother is not like to speak with me. ...,even my brother is not like to speak with me. ...
7,0,As per your request 'Melle Melle (Oru Minnamin...,as per your request 'melle melle (oru minnamin...
8,1,WINNER!! As a valued network customer you have...,winner!! as a valued network customer you have...
9,1,Had your mobile 11 months or more? U R entitle...,had your mobile 11 months or more? u r entitle...


In [191]:
#stopword removal
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\baska\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [192]:
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

df["text_wo_stop"] = df["text_lower"].apply(lambda text: remove_stopwords(text))
df.head()

,spam,message,text_lower,text_wo_stop
0,0,"Go until jurong point, crazy.. Available only ...","go until jurong point, crazy.. available only ...","go jurong point, crazy.. available bugis n gre..."
1,0,Ok lar... Joking wif u oni...,ok lar... joking wif u oni...,ok lar... joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in 2 a wkly comp to win fa cup fina...,free entry 2 wkly comp win fa cup final tkts 2...
3,0,U dun say so early hor... U c already then say...,u dun say so early hor... u c already then say...,u dun say early hor... u c already say...
4,0,"Nah I don't think he goes to usf, he lives aro...","nah i don't think he goes to usf, he lives aro...","nah think goes usf, lives around though"


In [193]:
#stemming didnt workout cause it generated lot of meaningless and useless words for NLP Processing instead apporached with lemmatization

In [194]:
#lemmatization
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

df["text_lemmatized"] = df["text_wo_stop"].apply(lambda text: lemmatize_words(text))
df.head()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\baska\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,spam,message,text_lower,text_wo_stop,text_lemmatized
0,0,"Go until jurong point, crazy.. Available only ...","go until jurong point, crazy.. available only ...","go jurong point, crazy.. available bugis n gre...","go jurong point, crazy.. available bugis n gre..."
1,0,Ok lar... Joking wif u oni...,ok lar... joking wif u oni...,ok lar... joking wif u oni...,ok lar... joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in 2 a wkly comp to win fa cup fina...,free entry 2 wkly comp win fa cup final tkts 2...,free entry 2 wkly comp win fa cup final tkts 2...
3,0,U dun say so early hor... U c already then say...,u dun say so early hor... u c already then say...,u dun say early hor... u c already say...,u dun say early hor... u c already say...
4,0,"Nah I don't think he goes to usf, he lives aro...","nah i don't think he goes to usf, he lives aro...","nah think goes usf, lives around though","nah think go usf, life around though"


In [195]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   spam             5572 non-null   int64 
 1   message          5572 non-null   object
 2   text_lower       5572 non-null   object
 3   text_wo_stop     5572 non-null   object
 4   text_lemmatized  5572 non-null   object
dtypes: int64(1), object(4)
memory usage: 217.8+ KB


In [196]:
#dropping other attributes
df=df.drop(columns=['text_lower','text_wo_stop'])
df.head(20)

,spam,message,text_lemmatized
0,0,"Go until jurong point, crazy.. Available only ...","go jurong point, crazy.. available bugis n gre..."
1,0,Ok lar... Joking wif u oni...,ok lar... joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...
3,0,U dun say so early hor... U c already then say...,u dun say early hor... u c already say...
4,0,"Nah I don't think he goes to usf, he lives aro...","nah think go usf, life around though"
5,1,FreeMsg Hey there darling it's been 3 week's n...,freemsg hey darling 3 week's word back! i'd li...
6,0,Even my brother is not like to speak with me. ...,even brother like speak me. treat like aid pat...
7,0,As per your request 'Melle Melle (Oru Minnamin...,per request 'melle melle (oru minnaminunginte ...
8,1,WINNER!! As a valued network customer you have...,winner!! valued network customer selected rece...
9,1,Had your mobile 11 months or more? U R entitle...,mobile 11 month more? u r entitled update late...


In [197]:
#TRAIN TEST SPLIT
X=df['text_lemmatized']
y=df['spam']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,random_state=30)

In [198]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

#THIS PROHJECT DIDNT NEEDED MUCH EDA APPROACHES 

MULTINOMIAL NAIVE BAYES CLASSIFIER - APPROACH 1

In [199]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_vectorized, y_train)
y_pred = nb_classifier.predict(X_test_vectorized)
accuracy = accuracy_score(y_test, y_pred)
confusion_mat = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)
print("Accuracy:", accuracy)
print("Confusion Matrix:\n", confusion_mat)
print("Classification Report:\n", classification_rep)

Accuracy: 0.9856502242152466
Confusion Matrix:
 [[964   6]
 [ 10 135]]
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99       970
           1       0.96      0.93      0.94       145

    accuracy                           0.99      1115
   macro avg       0.97      0.96      0.97      1115
weighted avg       0.99      0.99      0.99      1115



MNB WITH  k-fold cross-validation - APPROACH 2

In [200]:
from sklearn.model_selection import cross_val_score, KFold
k = 4
kf = KFold(n_splits=k, shuffle=True, random_state=42)
cross_val_scores = cross_val_score(nb_classifier, X_train_vectorized, y_train, cv=kf)
print("Cross-Validation Scores:", cross_val_scores)
print("Mean Cross-Validation Score:", cross_val_scores.mean())
print("Standard Deviation:", cross_val_scores.std())



Cross-Validation Scores: [0.98116592 0.97845601 0.98114901 0.98204668]
Mean Cross-Validation Score: 0.9807044062120102
Standard Deviation: 0.0013479271394122375


In [201]:
#OVERFITTING DUE TO LACK OF TESTING DATA BUT TESTED WITH REAL TIME MESSAGES 

PROJECT TESTING 

In [202]:
user_input = "Nah I don't think he goes to usf"
user_input_vectorized = vectorizer.transform([user_input])
prediction = nb_classifier.predict(user_input_vectorized)
if prediction[0] == 'spam':
    print("The text message is classified as spam.")
else:
    print("The text message is not spam.")

The text message is not spam.
